In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# import re
# import gc
import codecs
# import matplotlib.pyplot as plt
# import seaborn as sns
# import tensorflow as tf
# from bs4 import BeautifulSoup
# from nltk.corpus import stopwords
# from keras.preprocessing.text import Tokenizer

# %matplotlib inline
%load_ext autotime

# pal = sns.color_palette()

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.86 ms


In [4]:
# Paths

if os.path.isdir('data'):
    QUORA_DATA_DIR = "data/"
    GLOVE_DATA_DIR = "data/"
else:
    QUORA_DATA_DIR = "/opt/datasets/quora/"
    GLOVE_DATA_DIR = "/opt/datasets/glove/"

TRAIN_CSV = QUORA_DATA_DIR + 'train.csv'
TEST_CSV = QUORA_DATA_DIR + 'test.csv'

glove_840B_300d = GLOVE_DATA_DIR + 'glove.840B.300d.txt'
GLOVE_DATA_FILE = glove_840B_300d

time: 5.55 ms


In [5]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = 45
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.01

time: 2.61 ms


# Data Analysis

In [6]:
df_train = pd.read_csv(TRAIN_CSV)
df_test = pd.read_csv(TEST_CSV)

# Train Data
train_feature_1_string = pd.Series(df_train['question1'].tolist()).astype(str)
train_feature_2_string = pd.Series(df_train['question2'].tolist()).astype(str)

target = pd.Series(df_train['is_duplicate'].tolist())

all_train_qs = train_feature_1_string + train_feature_2_string

# Test Data
test_feature_1_string = pd.Series(df_test['question1'].tolist()).astype(str)
test_feature_2_string = pd.Series(df_test['question2'].tolist()).astype(str)

all_test_qs = test_feature_1_string + test_feature_2_string

all_qs = all_train_qs + all_test_qs

print(all_train_qs.tolist()[:10])

df_train.head()

['What is the step by step guide to invest in share market in india?What is the step by step guide to invest in share market?', 'What is the story of Kohinoor (Koh-i-Noor) Diamond?What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?', 'How can I increase the speed of my internet connection while using a VPN?How can Internet speed be increased by hacking through DNS?', 'Why am I mentally very lonely? How can I solve it?Find the remainder when [math]23^{24}[/math] is divided by 24,23?', 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?Which fish would survive in salt water?', "Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?", 'Should I buy tiago?What keeps childern active and far from phone and video games?', 'How can I be a good geologist?What should I do to be a great geologist?', 'When do you us

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


time: 20.1 s


In [7]:
df_test.head()

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


time: 11.6 ms


# Text Analysis

In [8]:
all_qids = df_train.qid1 + df_train.qid2

train_qs = df_train.question1 + df_train.question2

time: 275 ms


In [9]:
total_ques_pairs = len(df_train)
print('Total number of question pairs for training: {}'.format(total_ques_pairs))

duplicate_ques_pairs = round(df_train['is_duplicate'].mean()*100, 2)
print('Duplicate pairs: {}%'.format(duplicate_ques_pairs))

unique_qids = len(np.unique(train_qs.fillna("")))
print('Total number of questions in the training data: {}'.format(unique_qids))

print('Number of questions that appear multiple times: {}'.format(np.sum(all_qids.value_counts() > 1)))

print("Total number of questions in Quora dataset: {}".format(len(all_qs)))

Total number of question pairs for training: 404290
Duplicate pairs: 36.92%
Total number of questions in the training data: 404289
Number of questions that appear multiple times: 50205
Total number of questions in Quora dataset: 2345796
time: 1.19 s


## Text Processing

* TODO: Check the AVG length of english words and filter them out in GLOVE DATA
* TODO: Check the AVG length of english words and filter them out in INPUT DATA


In [65]:
import string

ACC_ALPHA_NUM_CHARS = set(map(ord, string.ascii_lowercase +  string.digits + ' ')) # adding space here
ACC_SPECIAL_CHARS = set(map(ord, '''~!@#$%^&*()_+`-=[]\{}|;':"<>?,./<\/>|?'"+=-*&^%$#@!_.;:{}()[]~!@#$%^&*()_+''')) 

def text_processor(text):
    """Text processing based on globally defined ACC_* variables.

    Args:
        text(string): input string

    Features:
        * clears trailing or heading space or tabs
        * converts text to lower case
        * adds appends and prepends spaces to all special charecters
    """
    new_text = []
    new_text_append = new_text.append
    for char in text.strip().lower():
        if ord(char) in ACC_ALPHA_NUM_CHARS:
            new_text_append(char)
        elif ord(char) in ACC_SPECIAL_CHARS:
            new_text_append(" " + char + " ")
    return ''.join(new_text)

time: 8.15 ms


In [66]:
import codecs

def glove_embeddings():
#     limit = 100000
    print('Indexing word vectors.')
    embeddings_index = dict()
    with codecs.open(GLOVE_DATA_FILE, encoding='utf-8') as f:
        for line in f:
#             Line expected format '<key> <vector coeffecients>'
#              Example 'A 0.2341 0.12313 0.31432 0.123414 ....'
            key, *vector = line.split(' ')
            if len(key) == 1 and (ord(key) in ACC_ALPHA_NUM_CHARS or ord(key) in ACC_SPECIAL_CHARS):
#                 single charecter
                embeddings_index[key] = np.asarray(vector, dtype='float32')
            elif key.islower() and len(key) > 1 and key.isalpha():
#                 text words - no symbols
                embeddings_index[key] = np.asarray(vector, dtype='float32')
#             # stopper key word
#             limit -= 1
#             if limit < 1:
#                 break
    return embeddings_index

time: 11.2 ms


In [67]:
if ('embeddings_index' not in dir()):
    print('Indexing word vectors.')
    embeddings_index = glove_embeddings()
    print('Found %s word vectors.' % len(embeddings_index))
else:
    print('Skipped to save some time!')

Skipped to save some time!
time: 3.65 ms


In [10]:
words_corpus = ''

for msg in all_test_qs:
    words_corpus += ' '
    words_corpus += msg
    
words_corpus = set(words_corpus.split())

time: 14.4 s


In [11]:
words_corpus = ' '.join(words_corpus)

special_chars = set([ i for i in words_corpus if not i.isalpha()])

time: 1.56 s


In [14]:
len(special_chars)

262

time: 3.49 ms


In [15]:
msg

'What is the best medication equation erectile dysfunction?How do I out get rid of Erectile Dysfunction?'

time: 3.68 ms


time: 7.68 ms


In [87]:
msg = 'What is the best medication equation erectile dysfunction?How do I out get rid of Erectile Dysfunction?'

msg, text_processor(msg)

('What is the best medication equation erectile dysfunction?How do I out get rid of Erectile Dysfunction?',
 'what is the best medication equation erectile dysfunction ? how do i out get rid of erectile dysfunction ? ')

time: 4.31 ms


In [ ]:
l = ["just say a word","of some length"]
ll = list(map(lambda l: l.split(" "), l))
seq_length = 5
ll
e = {"just": [0,0,0,0,1],
     "say" : [0,0,0,1,1],
     "a" : [0,1,1,1,0],
     "word" : [1,0,0,1,0],
     "of" : [0,1,0,1,0],
     "some" : [0,1,0,1,0],
     "length" : [0,0,1,1,0],
     '<PAD>': [0,0,0,0,0]
    }

In [ ]:
# list of words --> lows

from pprint import pprint as pp

def lows_padding(list_of_words, seq_length=5, append=True):
    """Pads/slices given bag of words for specified length."""
    if len(list_of_words) == seq_length:
        return list_of_words
    if len(list_of_words) > seq_length:
        return list_of_words[:seq_length]
    
    tmp = ['<PAD>' for i in range(seq_length - len(list_of_words))]
    if append:
        return list_of_words + tmp
    return  tmp + list_of_words

def lows_embedding(list_of_words, serializer):
    """To serializer/string Tokenize the list of words."""
    return list(map(lambda x: serializer[x], list_of_words))

def lows_transformer(list_of_words, serializer, seq_length, append):
    """To pad the given list of words and serialiase them."""
    return lows_embedding(lows_padding(list_of_words, seq_length, append),
                          serializer=e)

bag_of_lows_transformer = lambda list_of_words: lows_transformer(list_of_words, serializer=e,
                                                                 seq_length=4, append=True)

pp(list(map(bag_of_lows_transformer, ll)))

In [ ]:
ll

In [ ]:
bag_of_lows_transformer(['just', 'say', 'a', 'word'])

In [ ]:
bag = list(map(bag_of_lows_transformer, ll))

In [ ]:
def get_feature(embed_dict, list_of_strings, max_seq_length, embed_dim):
    features = np.zeros((len(list_of_strings), max_seq_length, embed_dim), dtype=float)
    list_of_bag_of_words = map(lambda l: list_of_words_padding(list_of_words=l.split(),
                                                                seq_length=max_seq_length,
                                                                append=True),
                                list_of_strings)
    return list_of_bag_of_words

list(get_feature(e, l, 7, 5))

In [ ]:
sample = df_train.head()

In [ ]:
sample.question1

# Build the Graph

In [ ]:
lstm_size = 256
lstm_layers = 1
batch_size = 250
learning_rate = 0.001

In [ ]:
# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, None, None], name='inputs') #[Number of ques, Seq/Ques Length, Embed Dims]
    labels_ = tf.placeholder(tf.float32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

http://suriyadeepan.github.io/2016-06-28-easy-seq2seq/

https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words

https://github.com/yuhaozhang/sentence-convnet

In [ ]:
500 / 60